In [5]:
"""
Exoplanet Habitability Analysis
Searching for Earth 2.0 candidates in NASA's Exoplanet Archive

Author: Alan Geirnaert
Dataset: NASA Exoplanet Archive (February 2026)
Challenge: Codédex Data Analysis Monthly Challenge
"""

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Color scheme for visualizations
COLORS = {
    'bg_dark': '#0a0a0f',
    'accent_blue': '#00d4ff',
    'accent_purple': '#a855f7',
    'accent_pink': '#ec4899',
    'accent_green': '#10b981',
    'accent_yellow': '#fbbf24',
    'accent_orange': '#f97316',
    'accent_red': '#ef4444',
    'text_light': '#e2e8f0',
    'goldilocks': '#22c55e',
    'earth_blue': '#3b82f6',
}

print("Libraries loaded successfully")

Libraries loaded successfully


In [6]:
# Load NASA Exoplanet Archive data
print("\nLoading NASA Exoplanet Archive data...")

df_raw = pd.read_csv('./exoplanets.csv', comment='#', low_memory=False)

print(f"Total entries in archive: {len(df_raw):,}")
print("Note: Many planets have multiple measurement rows from different sources")
print("Consolidating to get best estimates...\n")

# Data Consolidation Strategy
# Using median values from all measurements per planet to maximize accuracy

# Columns for aggregation - numeric use median, categorical use first non-null

numeric_cols = [
    'pl_rade',      # Planet radius (Earth radii)
    'pl_bmasse',    # Planet mass (Earth masses)
    'pl_eqt',       # Equilibrium temperature (K)
    'pl_insol',     # Insolation flux (Earth flux)
    'pl_orbper',    # Orbital period (days)
    'pl_orbsmax',   # Orbital semi-major axis (AU)
    'pl_orbeccen',  # Orbital eccentricity
    'st_teff',      # Stellar effective temperature (K)
    'st_rad',       # Stellar radius (Solar radii)
    'st_mass',      # Stellar mass (Solar masses)
    'st_lum',       # Stellar luminosity (log Solar)
    'sy_dist',      # Distance to system (parsecs)
]

categorical_cols = [
    'hostname',          # Host star name
    'st_spectype',       # Stellar spectral type
    'discoverymethod',   # Discovery method
    'disc_facility',     # Discovery facility
]

# Keep track of discovery year (take minimum = first discovery)
date_cols = ['disc_year']

def consolidate_planet_data(group):
    """Combine multiple measurements using median aggregation."""
    result = {'pl_name': group.name}
    
    # Numeric columns: median of all non-null values
    for col in numeric_cols:
        if col in group.columns:
            values = group[col].dropna()
            if len(values) > 0:
                result[col] = values.median()
                result[f'{col}_count'] = len(values)  # Track how many measurements
            else:
                result[col] = np.nan
                result[f'{col}_count'] = 0
    
    # Categorical columns: first non-null value
    for col in categorical_cols:
        if col in group.columns:
            non_null = group[col].dropna()
            result[col] = non_null.iloc[0] if len(non_null) > 0 else np.nan
    
    # Discovery year: earliest
    if 'disc_year' in group.columns:
        years = group['disc_year'].dropna()
        result['disc_year'] = int(years.min()) if len(years) > 0 else np.nan
    
    # Count how many source rows we had
    result['source_row_count'] = len(group)
    
    return pd.Series(result)

print("Consolidating data by planet name...")

# Group by planet name and consolidate
df_unique = df_raw.groupby('pl_name').apply(consolidate_planet_data).reset_index(drop=True)
total_planets = len(df_unique)

print(f"Consolidated into {total_planets:,} unique confirmed exoplanets\n")

# Consolidation statistics
multi_source = df_unique[df_unique['source_row_count'] > 1]

print(f"Planets with multiple measurement sources: {len(multi_source):,}")
print(f"Average measurements per planet: {df_unique['source_row_count'].mean():.1f}")
print(f"Max measurements for single planet: {df_unique['source_row_count'].max()}")

# Example of a planet that benefited from consolidation
if len(multi_source) > 0:
    example = multi_source.sort_values('source_row_count', ascending=False).iloc[0]
    print(f"Example: {example['pl_name']} had {int(example['source_row_count'])} measurement rows")
    
    # Show key consolidated metrics
    if pd.notna(example.get('pl_rade_count', 0)) and example.get('pl_rade_count', 0) > 1:
        print(f"  - Radius: median of {int(example['pl_rade_count'])} measurements")    
    if pd.notna(example.get('pl_eqt_count', 0)) and example.get('pl_eqt_count', 0) > 1:
        print(f"  - Temperature: median of {int(example['pl_eqt_count'])} measurements")

print("\nConsolidation complete\n")

df_unique.head()


Loading NASA Exoplanet Archive data...
Total entries in archive: 39,315
Note: Many planets have multiple measurement rows from different sources
Consolidating to get best estimates...

Consolidating data by planet name...
Consolidated into 6,100 unique confirmed exoplanets

Planets with multiple measurement sources: 5,211
Average measurements per planet: 6.4
Max measurements for single planet: 34
Example: TrES-2 b had 34 measurement rows
  - Radius: median of 26 measurements
  - Temperature: median of 13 measurements

Consolidation complete



,pl_name,pl_rade,pl_rade_count,pl_bmasse,pl_bmasse_count,pl_eqt,pl_eqt_count,pl_insol,pl_insol_count,pl_orbper,...,st_lum,st_lum_count,sy_dist,sy_dist_count,hostname,st_spectype,discoverymethod,disc_facility,disc_year,source_row_count
0,11 Com b,NaN,0,5434.700000,3,NaN,0,NaN,0,324.620000,...,2.110615,2,93.1846,3,11 Com,G8 III,Radial Velocity,Xinglong Station,2007.0,3
1,11 UMi b,NaN,0,3432.400000,3,NaN,0,NaN,0,516.219985,...,NaN,0,125.3210,3,11 UMi,K4 III,Radial Velocity,Thueringer Landessternwarte Tautenburg,2009.0,3
2,14 And b,NaN,0,1131.151301,3,NaN,0,NaN,0,186.300000,...,1.801460,2,75.4392,3,14 And,K0 III,Radial Velocity,Okayama Astrophysical Observatory,2008.0,3
3,14 Her b,NaN,0,1523.958887,8,NaN,0,NaN,0,1766.410000,...,-0.151365,2,17.9323,8,14 Her,K0 V,Radial Velocity,W. M. Keck Observatory,2002.0,8
4,16 Cyg B b,NaN,0,533.942200,6,NaN,0,NaN,0,799.475000,...,0.097290,1,21.1397,6,16 Cyg B,G2.5 V,Radial Velocity,Multiple Observatories,1996.0,6


In [7]:
# Habitability Criteria ("Goldilocks Zone")
# Based on conservative estimates for Earth-like conditions
print("Defining habitability criteria...")
print("Size: 0.5-1.6 Earth radii (rocky planet range)")
print("Temperature: 180-310K (liquid water zone)")
print("Star type: F, G, or K spectral classes (stable, long-lived)\n")

# Thresholds
RADIUS_MIN, RADIUS_MAX = 0.5, 1.6
TEMP_MIN, TEMP_MAX = 180, 310
INSOL_MIN, INSOL_MAX = 0.2, 2.0

Defining habitability criteria...
Size: 0.5-1.6 Earth radii (rocky planet range)
Temperature: 180-310K (liquid water zone)
Star type: F, G, or K spectral classes (stable, long-lived)



In [8]:
# Apply filters step by step
print("Applying habitability filters...\n")

# Filter functions
def is_habitable_temp(row):
    """Check if planet is in habitable temperature range."""
    # Use either equilibrium temp or insolation flux as proxy
    if pd.notna(row['pl_eqt']) and TEMP_MIN <= row['pl_eqt'] <= TEMP_MAX:
        return True
    if pd.notna(row['pl_insol']) and INSOL_MIN <= row['pl_insol'] <= INSOL_MAX:
        return True
    return False

def is_sunlike_star(spec_type):
    """F, G, K spectral types only."""
    if pd.isna(spec_type):
        return False
    spec_str = str(spec_type).upper().strip()
    return spec_str and spec_str[0] in ['F', 'G', 'K']

# Build filter funnel
funnel_data = []
funnel_data.append(('All Confirmed Exoplanets', total_planets))

stage1 = df_unique[df_unique['pl_rade'].notna()].copy()
funnel_data.append(('With Radius Measurement', len(stage1)))

stage2 = stage1[(stage1['pl_rade'] >= RADIUS_MIN) & (stage1['pl_rade'] <= RADIUS_MAX)].copy()
funnel_data.append(('Rocky Size (0.5-1.6 R⊕)', len(stage2)))

stage3 = stage2[stage2['pl_eqt'].notna() | stage2['pl_insol'].notna()].copy()
funnel_data.append(('With Temperature Data', len(stage3)))

stage3['is_habitable'] = stage3.apply(is_habitable_temp, axis=1)
stage4 = stage3[stage3['is_habitable']].copy()
funnel_data.append(('Habitable Temperature', len(stage4)))

stage4['sunlike_star'] = stage4['st_spectype'].apply(is_sunlike_star)
stage5 = stage4[stage4['sunlike_star']].copy()
funnel_data.append(('Sun-like Star (F, G, K)', len(stage5)))

earth_candidates = stage5.copy()

# Print filtering results
for i, (stage, count) in enumerate(funnel_data):
    if i == 0:
        print(f"Starting pool: {count:,} planets")
    else:
        prev_count = funnel_data[i-1][1]
        lost = prev_count - count
        print(f"  {stage}: {count:,} ({lost:,} eliminated)")

print(f"\nFinal Earth 2.0 candidates: {len(earth_candidates)}")
print()

Applying habitability filters...

Starting pool: 6,100 planets
  With Radius Measurement: 4,579 (1,521 eliminated)
  Rocky Size (0.5-1.6 R⊕): 1,136 (3,443 eliminated)
  With Temperature Data: 1,080 (56 eliminated)
  Habitable Temperature: 30 (1,050 eliminated)
  Sun-like Star (F, G, K): 2 (28 eliminated)

Final Earth 2.0 candidates: 2



In [9]:
# Visualization 1: Funnel chart showing filter progression

stages = [s[0] for s in funnel_data]
values = [s[1] for s in funnel_data]

fig = go.Figure(go.Funnel(
    y=stages,
    x=values,
    textposition="inside",
    textinfo="value+percent initial",
    opacity=0.85,
    marker=dict(
        color=[COLORS['accent_blue'], COLORS['accent_purple'], COLORS['accent_pink'],
               COLORS['accent_orange'], COLORS['accent_yellow'], COLORS['goldilocks']],
        line=dict(width=2, color="white")
    ),
    connector=dict(line=dict(color="white", width=1))
))

fig.update_layout(
    title=dict(text="Exoplanet Habitability Filter Funnel<br><sup>Searching for Earth 2.0 Candidates</sup>",
               font=dict(size=24, color=COLORS['text_light'])),
    paper_bgcolor=COLORS['bg_dark'],
    plot_bgcolor=COLORS['bg_dark'],
    font=dict(color=COLORS['text_light']),
    height=600
)
fig.show()

In [10]:
# Visualization 2: Size distribution histogram

df_radius = df_unique[df_unique['pl_rade'].notna()].copy()

# Separate by category
rocky = df_radius[(df_radius['pl_rade'] >= RADIUS_MIN) & (df_radius['pl_rade'] <= RADIUS_MAX)]['pl_rade']
sub_earth = df_radius[df_radius['pl_rade'] < RADIUS_MIN]['pl_rade']
super_earth = df_radius[df_radius['pl_rade'] > RADIUS_MAX]['pl_rade']

fig = go.Figure()

# Add histogram traces
fig.add_trace(go.Histogram(x=super_earth, name='Super-Earth/Gas', 
                           marker_color=COLORS['accent_purple'], opacity=0.7,
                           xbins=dict(start=0.1, end=30, size=0.3)))
fig.add_trace(go.Histogram(x=rocky, name='Rocky (Goldilocks)', 
                           marker_color=COLORS['goldilocks'], opacity=0.9,
                           xbins=dict(start=0.1, end=30, size=0.3)))
fig.add_trace(go.Histogram(x=sub_earth, name='Sub-Earth', 
                           marker_color=COLORS['accent_blue'], opacity=0.7,
                           xbins=dict(start=0.1, end=30, size=0.3)))

# Add Earth reference line
fig.add_vline(x=1.0, line_dash="dash", line_color=COLORS['earth_blue'], 
              annotation_text="Earth", annotation_position="top")

# Add Goldilocks zone shading
fig.add_vrect(x0=RADIUS_MIN, x1=RADIUS_MAX, fillcolor=COLORS['goldilocks'], 
              opacity=0.1, line_width=0, annotation_text="Goldilocks Zone")

fig.update_layout(
    title='Exoplanet Size Distribution<br><sup>Rocky Planets vs Gas Giants</sup>',
    xaxis_title='Planet Radius (Earth Radii)',
    yaxis_title='Number of Planets',
    barmode='overlay',
    paper_bgcolor=COLORS['bg_dark'],
    plot_bgcolor=COLORS['bg_dark'],
    font=dict(color=COLORS['text_light']),
    legend=dict(title="Size Category"),
    height=500,
    xaxis_type="log"
)
fig.show()

In [11]:
# Visualization 3: Temperature vs size scatter plot

df_temp = df_unique[(df_unique['pl_rade'].notna()) & (df_unique['pl_eqt'].notna())].copy()
df_temp['in_goldilocks'] = (
    (df_temp['pl_rade'] >= RADIUS_MIN) & (df_temp['pl_rade'] <= RADIUS_MAX) & 
    (df_temp['pl_eqt'] >= TEMP_MIN) & (df_temp['pl_eqt'] <= TEMP_MAX))

fig = px.scatter(df_temp, x='pl_eqt', y='pl_rade', 
                 color='in_goldilocks',
                 color_discrete_map={True: COLORS['goldilocks'], False: COLORS['accent_purple']},
                 hover_name='pl_name',
                 hover_data={'pl_eqt': ':.0f', 'pl_rade': ':.2f', 'hostname': True, 'disc_year': True},
                 opacity=0.6,
                 title='Habitability Map: Temperature vs Size',
                 labels={'pl_eqt': 'Equilibrium Temperature (K)', 'pl_rade': 'Planet Radius (Earth Radii)'})

# Add Goldilocks zone box
fig.add_shape(type="rect", x0=TEMP_MIN, x1=TEMP_MAX, y0=RADIUS_MIN, y1=RADIUS_MAX,
              line=dict(color=COLORS['goldilocks'], width=3, dash="dash"),
              fillcolor=COLORS['goldilocks'], opacity=0.1)

# Add Earth marker
fig.add_trace(go.Scatter(x=[255], y=[1.0], mode='markers+text',
                         marker=dict(size=15, color=COLORS['earth_blue'], symbol='star'),
                         text=['Earth'], textposition='top center',
                         name='Earth', showlegend=True))

fig.update_layout(
    paper_bgcolor=COLORS['bg_dark'],
    plot_bgcolor=COLORS['bg_dark'],
    font=dict(color=COLORS['text_light']),
    yaxis_type="log",
    height=600,
    legend_title="In Goldilocks Zone?"
)
fig.show()

In [12]:
# Visualization 4: Discovery timeline

yearly = df_unique.groupby('disc_year').size().reset_index(name='count')
yearly_rocky = df_unique[(df_unique['pl_rade'] >= RADIUS_MIN) & 
                         (df_unique['pl_rade'] <= RADIUS_MAX)].groupby('disc_year').size().reset_index(name='rocky')

yearly = yearly.merge(yearly_rocky, on='disc_year', how='left').fillna(0)
yearly['cumulative'] = yearly['count'].cumsum()

fig = make_subplots(rows=2, cols=1, subplot_titles=('Discoveries Per Year', 'Cumulative Growth'),
                    vertical_spacing=0.12)

fig.add_trace(go.Bar(x=yearly['disc_year'], y=yearly['count'], name='All Planets',
                     marker_color=COLORS['accent_purple'], opacity=0.7), row=1, col=1)
fig.add_trace(go.Bar(x=yearly['disc_year'], y=yearly['rocky'], name='Rocky Planets',
                     marker_color=COLORS['goldilocks'], opacity=0.9), row=1, col=1)

fig.add_trace(go.Scatter(x=yearly['disc_year'], y=yearly['cumulative'], mode='lines+markers',
                         name='Total Discovered', line=dict(color=COLORS['accent_purple'], width=3),
                         fill='tozeroy', fillcolor=f"rgba(168, 85, 247, 0.2)"), row=2, col=1)

# Mission markers
for year, name, color in [(2009, 'Kepler', COLORS['accent_yellow']), 
                          (2018, 'TESS', COLORS['accent_blue']), 
                          (2022, 'JWST', COLORS['accent_orange'])]:
    fig.add_vline(x=year, line_dash="dash", line_color=color, row=1, col=1,
                  annotation_text=name, annotation_position="top")

fig.update_layout(
    title=dict(text="Exoplanet Discovery Timeline", font=dict(size=22)),
    paper_bgcolor=COLORS['bg_dark'],
    plot_bgcolor=COLORS['bg_dark'],
    font=dict(color=COLORS['text_light']),
    height=700,
    barmode='overlay',
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)
fig.show()

In [13]:
# Visualization 5: Candidate profiles

candidates = earth_candidates[['pl_name', 'pl_rade', 'pl_eqt', 'st_spectype', 
                                   'sy_dist', 'disc_year', 'discoverymethod']].copy()

fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"}, {"type": "bar"}],
                           [{"type": "pie"}, {"type": "bar"}]],
                    subplot_titles=("Candidate Sizes", "Candidate Temperatures",
                                   "Discovery Methods", "Host Star Types"))

# Sizes
fig.add_trace(go.Bar(y=candidates['pl_name'], x=candidates['pl_rade'], orientation='h',
                     marker_color=COLORS['goldilocks'], name='Radius'), row=1, col=1)
fig.add_vline(x=1.0, line_dash="dash", line_color=COLORS['earth_blue'], row=1, col=1)

# Temperatures
temp_data = candidates[candidates['pl_eqt'].notna()]
fig.add_trace(go.Bar(y=temp_data['pl_name'], x=temp_data['pl_eqt'], orientation='h',
                     marker_color=COLORS['accent_blue'], name='Temperature'), row=1, col=2)
fig.add_vline(x=255, line_dash="dash", line_color=COLORS['earth_blue'], row=1, col=2)

# Discovery methods
methods = candidates['discoverymethod'].value_counts()
fig.add_trace(go.Pie(labels=methods.index, values=methods.values,
                     marker_colors=[COLORS['accent_purple'], COLORS['accent_pink']]), row=2, col=1)

# Star types
stars = candidates['st_spectype'].apply(lambda x: str(x)[0] if pd.notna(x) else '?').value_counts()
fig.add_trace(go.Bar(x=stars.index, y=stars.values, 
                     marker_color=COLORS['accent_orange'], name='Star Type'), row=2, col=2)

fig.update_layout(
    title=dict(text=f"Earth 2.0 Candidate Analysis ({len(earth_candidates)} planets)", font=dict(size=24)),
    paper_bgcolor=COLORS['bg_dark'],
    plot_bgcolor=COLORS['bg_dark'],
    font=dict(color=COLORS['text_light']),
    height=700,
    showlegend=False
)
fig.show()

In [14]:
# Final Summary
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60 + "\n")

for stage, count in funnel_data:
    print(f"{stage}: {count:,}")

print()
print(f"\nFinal candidates: {len(earth_candidates)}")

if len(earth_candidates) > 0:
    print()
    print("\nCandidate details:")
    print("-" * 60)
    for _, row in earth_candidates.iterrows():
        dist_info = f", Distance: {row['sy_dist']:.1f} pc" if pd.notna(row['sy_dist']) else ""
        print(f"{row['pl_name']}:")
        print(f"  Radius: {row['pl_rade']:.2f} R_Earth")
        print(f"  Temperature: {row['pl_eqt']:.0f}K")
        print(f"  Host star: {row['st_spectype']}{dist_info}\n")


SUMMARY

All Confirmed Exoplanets: 6,100
With Radius Measurement: 4,579
Rocky Size (0.5-1.6 R⊕): 1,136
With Temperature Data: 1,080
Habitable Temperature: 30
Sun-like Star (F, G, K): 2


Final candidates: 2


Candidate details:
------------------------------------------------------------
Kepler-452 b:
  Radius: 1.50 R_Earth
  Temperature: 220K
  Host star: G2, Distance: 551.7 pc

Kepler-62 f:
  Radius: 1.44 R_Earth
  Temperature: 205K
  Host star: K2 V, Distance: 300.9 pc



In [15]:
# Analysis conclusion

percentage = (len(earth_candidates) / total_planets) * 100 if total_planets > 0 else 0
print(f"\nOf {total_planets:,} confirmed exoplanets, {len(earth_candidates)} ({percentage:.4f}%) ")
print(f"meet strict Earth-analog criteria.")
print("\nThis demonstrates the rarity of truly Earth-like worlds in our current dataset.")
print("Future missions (JWST, etc.) may help characterize these candidates further.\n")


Of 6,100 confirmed exoplanets, 2 (0.0328%) 
meet strict Earth-analog criteria.

This demonstrates the rarity of truly Earth-like worlds in our current dataset.
Future missions (JWST, etc.) may help characterize these candidates further.

